In [3]:
!pip install -U ortools pandas

Requirement already up-to-date: ortools in /home/maa/miniconda3/envs/pytorch/lib/python3.7/site-packages (7.2.6977)
     |████████████████████████████████| 10.1MB 7.4MB/s 


In [1]:
import numpy as np
import pandas as pd
from ortools.linear_solver import pywraplp

np.random.seed(12)
samples = 500
probability = 0.2

index = ['name', 'cost', 'size', 'handling', 'positioning', 'joining']
names = ['robot' + str(i) for i in range(samples)]
costs = np.random.randint(1000, 10000, size=(samples)).tolist()
handling = np.random.binomial(1, probability, samples)
positioning = np.random.binomial(1, probability, samples)
joining = np.random.binomial(1, probability, samples)
sizes = np.round(np.random.rand((samples)) * 10, 2)

df = pd.DataFrame({'name': names, 'cost': costs, 'size': sizes, 'handling': handling, 'positioning': positioning, 'joining': joining})
df.head(10)

,name,cost,size,handling,positioning,joining
0,robot0,6787,9.98,0,0,0
1,robot1,4325,0.42,1,0,1
2,robot2,8409,1.20,1,1,0
3,robot3,4714,8.71,0,0,0
4,robot4,1278,0.68,1,0,0
5,robot5,9241,8.38,0,0,1
6,robot6,4725,7.91,0,1,1
7,robot7,5569,0.43,0,0,1
8,robot8,9610,6.49,0,0,0
9,robot9,9651,4.94,1,0,0


In [2]:
constraints = {'cost': 20000, 'size': 10.0, 'max_num': 5, 'handling': 8, 'positioning': 5, 'joining': 3}

In [3]:
solver = pywraplp.Solver('simple_lp_programm', pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
solver

<ortools.linear_solver.pywraplp.Solver; proxy of <Swig Object of type 'operations_research::MPSolver *' at 0x7f9e691d9420> >

### Create Variables
for every robot in the `df` set one variable which corresponds to the number of occurences for this robot. At first every variable is 1, then it will be minimized.
[example](https://developers.google.com/optimization/mip/integer_opt)

In [4]:
solver.Clear()
# fitness = solver.NumVar(0, solver.infinity(), 'fitness')
sol = [[]] * len(df)
objective_num = solver.Objective()
for i in range(0, len(df)):
    sol[i] = solver.IntVar(0, solver.infinity(), df['name'][i])
    objective_num.SetCoefficient(sol[i], 1)
objective_num.SetMinimization()
# cost = solver.NumVar(0, constraints['max_cost'], 'cost')
# size = solver.NumVar(0, constraints['max_size'], 'size')
# num = solver.IntVar(0, constraints['max_num'], 'num')

print('Number of variables:', solver.NumVariables())

Number of variables: 500


### create constraints
necessary:

cost $c$, size $s$ and num $n$


\begin{equation}
0 \leq c \leq c_{max} \qquad \in [0..20000] \\
0 \leq s \leq s_{max} \qquad \in [0..10] \\
0 \leq n \leq n_{max} \qquad \in \{ 0..5 \}
\end{equation}

and the number of handling, positionig and joining resources must be:

\begin{equation}
h_{min} \leq h \\
p_{min} \leq p \\
j_{min} \leq j
\end{equation}

In [5]:
# create constraints
# cost_c = solver.Constraint(0, constraints['max_cost'], 'cost')
# size_c = solver.Constraint(0, constraints['max_size'], 'size')
# num_c = solver.Constraint(0, constraints['max_num'], 'num')

# cost_c.SetCoefficient(cost, 1)
# size_c.SetCoefficient(size, 1)
# num_c.SetCoefficient(num, 1)
names_min = ['handling', 'positioning', 'joining']
names_max = ['cost', 'size']

const = [0] * (len(names_max) + len(names_min))
for i in range(len(names_max)):
    const[i] = solver.Constraint(0, constraints[names_max[i]], names_max[i])
    for j in range(len(df)):
        const[i].SetCoefficient(sol[j], float(df[names_max[i]][j]))
        

for i in range(len(names_min)):
    const[i] = solver.Constraint(constraints[names_min[i]], solver.infinity(), names_min[i])
    for j in range(len(df)):
        const[i].SetCoefficient(sol[j], float(df[names_min[i]][j]))

print('Number of constraints:', solver.NumConstraints())

Number of constraints: 5


In [6]:
result_status = solver.Solve()
# The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL

# The solution looks legit (when using solvers others than
# GLOP_LINEAR_PROGRAMMING, verifying the solution is highly recommended!).
assert solver.VerifySolution(1e-7, True)

print('Solution:')
print('Number of robots:', objective_num.Value())

cost = 0
size = 0
handling = 0
positioning = 0
joining = 0
robots = []
for i in range(len(df)):
    if sol[i].solution_value() > 0: 
        c = df.loc[df.name==str(sol[i]), 'cost'].item()
        s = df.loc[df.name==str(sol[i]), 'size'].item()
        handling += df.loc[df.name==str(sol[i]), 'handling'].item() * sol[i].solution_value()
        positioning += df.loc[df.name==str(sol[i]), 'positioning'].item() * sol[i].solution_value()
        joining += df.loc[df.name==str(sol[i]), 'joining'].item() * sol[i].solution_value()
        robots.append(str(sol[i]))
        print('{:<10} | num: {:<5} | cost: {:<5} € | size: {:<5} m2'.format(str(sol[i]), sol[i].solution_value(), c, s))
        cost += c
        size += s

print('Cost =', cost, '€')
print('Size =', round(size, 3), 'm2')
print('Handling =', handling, '/', constraints['handling'])
print('Positioning =', positioning, '/', constraints['positioning'])
print('Joining =', joining, '/', constraints['joining'])

Solution:
Number of robots: 9.0
robot4     | num: 4.0   | cost: 1278  € | size: 0.68  m2
robot190   | num: 3.0   | cost: 2816  € | size: 1.75  m2
robot224   | num: 1.0   | cost: 5401  € | size: 0.56  m2
robot477   | num: 1.0   | cost: 1034  € | size: 0.84  m2
Cost = 10529 €
Size = 3.83 m2
Handling = 8.0 / 8
Positioning = 5.0 / 5
Joining = 3.0 / 3


**Objective here is to minimize the number of resources**

In [7]:
df.loc[df.name.isin(robots), :]

,name,cost,size,handling,positioning,joining
4,robot4,1278,0.68,1,0,0
190,robot190,2816,1.75,1,1,1
224,robot224,5401,0.56,1,1,0
477,robot477,1034,0.84,0,1,0
